In [1]:
import os
import sys
%reload_ext autoreload
%autoreload

project_root = os.path.abspath(os.path.join('../../../'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import pandas as pd
from ml_workbench.src.data_loaders.BablTask1Loader import split_parse_data

X_train, y_train = split_parse_data('/Users/enrico.t/code/ml_workbench/' + 
  'data/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt')

X_test, y_test = split_parse_data('/Users/enrico.t/code/ml_workbench/' + 
 'data/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt')

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from ml_workbench.src.vectorizers.ColumnBowVectorizer import ColumnBowVectorizer

pipeline = Pipeline([
    ('bow', ColumnBowVectorizer()),
    ('clf', SVC()) # probability = True ?
])

parameters = {
    'clf__C': [1, 10, 100, 1000],
    'clf__gamma': [1, 0.1, 0.001, 0.0001], 
}

grid = GridSearchCV(
    pipeline, 
    parameters, 
    cv=5,
    n_jobs=-1, 
    verbose=1
)

grid.fit(X_train, y_train)
y_predict = grid.predict(X_test)

print grid.best_params_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   58.9s finished
/Users/enrico.t/code/ml_workbench/venv/lib/python2.7/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'clf__gamma': 0.0001, 'clf__C': 100}


In [4]:
from pandas_ml import ConfusionMatrix

cm = ConfusionMatrix(y_test.values.flatten(), y_predict)
cm.print_stats()
print cm.plot()

/Users/enrico.t/code/ml_workbench/venv/lib/python2.7/site-packages/pandas_ml/confusion_matrix/stats.py:60: FutureWarning: supplying multiple axes to axis is deprecated and will be removed in a future version.
  num = df[df > 1].dropna(axis=[0, 1], thresh=1).applymap(lambda n: choose(n, 2)).sum().sum() - np.float64(nis2 * njs2) / n2


Confusion Matrix:

Predicted  bathroom  bedroom  garden  hallway  kitchen  office  __all__
Actual                                                                 
bathroom         79       16      20       10       10      14      149
bedroom          23       83      21       11       16      17      171
garden           11       16     106       10       20      24      187
hallway          11       16      16       78       14      19      154
kitchen          14       11      17       12       92      11      157
office           27       15      19       15       17      89      182
__all__         165      157     199      136      169     174     1000


Overall Statistics:

Accuracy: 0.527
95% CI: (0.4955145839930299, 0.5583260325172703)
No Information Rate: ToDo
P-Value [Acc > NIR]: 6.635815403729728e-117
Kappa: 0.43163384241958164
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                 bathroom    bedroom    garden  \
Population               

In [7]:
bow = pipeline.named_steps['bow']
print bow._feature_names

[]
